### previously - workbook3 ;  in continuation - take preprocessSTG8_d4.csv
### Bert Prep
#### ---- set D4 -  Education, Educational

In [1]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_csv('preprocessSTG8_d4.csv')
df.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Calendar,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length
0,Calculus Tutorial 1: Introduction,com.RaySemiSoft.CalculusT1,Education,0.0,277,False,"In this tutorial, we first define what we mean...",https://sites.google.com/site/raysemisoft/home,0,0,...,0,0,0,0,0,0,0,1578,"in this tutorial, we first define what we mean...",1451
1,Chemistry Mission,academy.crow.chemistrymission,Educational,0.0,76,False,3 Game Modes:\r\nOrganic Chemistry\r\nInorgani...,https://crow.academy/privacy-policy.html,0,0,...,0,0,0,0,0,0,0,171,game modes organic chemistry inorganic chemist...,161


In [3]:
df.shape

(162047, 22)

In [4]:
df.dropna(subset=['Clean_Description'],inplace=True)

In [5]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [7]:
!pip install transformers

In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [9]:
import numpy as np

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# Record the length of each sequence (after truncating to 512)
lengths = []

print('Tokenizing descriptions...')

# For every sentence...
for sent in df.Clean_Description:

    # Report progress
    if ((len(input_ids) % 20000) == 0):
        print('   Read {:,} descriptions.'.format(len(input_ids)))

    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

    # Record the truncated length.
    lengths.append(len(encoded_sent))

print('DONE. ')
print('{:>10,} comments'.format(len(input_ids)))

Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors


Tokenizing descriptions...
   Read 0 descriptions.
   Read 20,000 descriptions.
   Read 40,000 descriptions.
   Read 60,000 descriptions.
   Read 80,000 descriptions.
   Read 100,000 descriptions.
   Read 120,000 descriptions.
   Read 140,000 descriptions.
   Read 160,000 descriptions.
DONE. 
   162,020 comments


In [10]:
print(lengths)
print(len(lengths))
print(df.shape)

[274, 29, 90, 91, 155, 96, 300, 22, 34, 236, 91, 53, 40, 89, 581, 219, 89, 98, 97, 6, 186, 129, 42, 726, 274, 18, 22, 48, 29, 13, 283, 89, 88, 109, 521, 179, 46, 355, 117, 344, 160, 650, 52, 282, 243, 28, 156, 18, 366, 487, 212, 21, 140, 362, 68, 232, 108, 11, 10, 337, 278, 48, 86, 91, 504, 138, 133, 100, 225, 254, 82, 163, 68, 343, 113, 354, 77, 117, 61, 92, 60, 241, 468, 277, 164, 79, 113, 479, 167, 100, 39, 263, 170, 598, 18, 95, 72, 295, 71, 38, 57, 631, 272, 61, 187, 564, 140, 68, 251, 54, 292, 98, 154, 164, 104, 59, 241, 44, 27, 69, 138, 261, 292, 128, 42, 619, 96, 75, 445, 709, 84, 180, 111, 145, 44, 24, 113, 317, 42, 640, 7, 65, 119, 95, 93, 147, 70, 328, 76, 428, 63, 93, 67, 72, 53, 117, 88, 38, 224, 44, 308, 83, 171, 344, 62, 619, 179, 70, 157, 63, 101, 37, 160, 358, 140, 78, 158, 98, 113, 102, 96, 99, 125, 71, 9, 92, 91, 439, 415, 50, 181, 510, 225, 12, 90, 384, 177, 115, 95, 175, 452, 631, 65, 8, 206, 105, 170, 586, 393, 47, 98, 59, 60, 417, 77, 62, 93, 157, 135, 49, 70, 35

In [11]:
df['token_length'] = lengths
df.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Calculus Tutorial 1: Introduction,com.RaySemiSoft.CalculusT1,Education,0.0,277,False,"In this tutorial, we first define what we mean...",https://sites.google.com/site/raysemisoft/home,0,0,...,0,0,0,0,0,0,1578,"in this tutorial, we first define what we mean...",1451,274
1,Chemistry Mission,academy.crow.chemistrymission,Educational,0.0,76,False,3 Game Modes:\r\nOrganic Chemistry\r\nInorgani...,https://crow.academy/privacy-policy.html,0,0,...,0,0,0,0,0,0,171,game modes organic chemistry inorganic chemist...,161,29


In [12]:
print(sum(df['token_length'] < 25))

11240


In [13]:
print(sum(df['token_length'] >= 510))

10760


In [14]:
print('    Min length: {:,} tokens'.format(min(lengths)))
print('    Min length: {:,} tokens'.format(max(lengths)))
print(' Median length: {:,} tokens'.format(np.median(lengths)))

    Min length: 3 tokens
    Min length: 1,399 tokens
 Median length: 124.0 tokens


In [15]:
df.head(3)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Calculus Tutorial 1: Introduction,com.RaySemiSoft.CalculusT1,Education,0.0,277,False,"In this tutorial, we first define what we mean...",https://sites.google.com/site/raysemisoft/home,0,0,...,0,0,0,0,0,0,1578,"in this tutorial, we first define what we mean...",1451,274
1,Chemistry Mission,academy.crow.chemistrymission,Educational,0.0,76,False,3 Game Modes:\r\nOrganic Chemistry\r\nInorgani...,https://crow.academy/privacy-policy.html,0,0,...,0,0,0,0,0,0,171,game modes organic chemistry inorganic chemist...,161,29
2,RACE ACADEMY,co.davos.snqkw,Education,0.0,186,False,RACE ACADEMY is an online platform for managin...,https://bit.ly/2YDTip0,0,1,...,1,0,1,1,0,0,503,race academy is an online platform for managin...,497,90


In [ ]:
## save the df dataset file - 

In [16]:
df.to_csv("preprocessSTG9_d4.csv",index=False)